# Peters and Taylor (2016)

> Total Q, and intangible capital measures from Peters and Taylor (2016)

In [ ]:
#| default_exp papers.peters_taylor_2016

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/peters-and-taylor-total-q/peters-and-taylor-total-q/'
LIBRARY = 'totalq'
TABLE = 'total_q'
LINK_LIBRARY = 'crsp'
LINK_TABLE = 'ccmxpf_lnkhist'
FREQ = 'A'
MIN_YEAR = 1950
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'permno'
ENTITY_ID_IN_CLEAN_DSET = 'permno'
TIME_VAR_IN_RAW_DSET = 'datadate'
TIME_VAR_IN_CLEAN_DSET = f'{FREQ}date'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS f`{LIBRARY}.{TABLE}` and `{LIBRARY}.{COMPANY_TABLE}`."

    try:
        db = wrds_api.Connection()
        funda = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
    finally:
        db.close()

    return funda[['name','type','wrds_library','wrds_table']].copy()

In [ ]:
#| eval: false
all_vars = list_all_vars()

In [ ]:
#| eval: false
all_vars

,name,type,wrds_library,wrds_table
0,gvkey,VARCHAR(6),totalq,total_q
1,datadate,DATE,totalq,total_q
2,fyear,DOUBLE_PRECISION,totalq,total_q
3,k_int_know,DOUBLE_PRECISION,totalq,total_q
4,k_int_org,DOUBLE_PRECISION,totalq,total_q
5,k_int_offbs,DOUBLE_PRECISION,totalq,total_q
6,k_int,DOUBLE_PRECISION,totalq,total_q
7,q_tot,DOUBLE_PRECISION,totalq,total_q


In [ ]:
#| export
def get_raw_data(
        vars: List[str]='*', # Default is to get all available variables
        required_vars: List[str]=['gvkey','datadate'], #list of variables that will get downloaded, even if not in `vars`
        nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
        start_date: str=None, # Start date in MM/DD/YYYY format
        end_date: str=None #End date in MM/DD/YYYY format
) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `{LIBRARY}.{TABLE}` library and adds PERMNO and PERMCO as in CCM"""
 
    wrds_api.validate_dates([start_date, end_date])

    sql_string=f"""SELECT c.lpermno as permno, c.lpermco as permco, c.liid, c.linkprim as linkprim, 
                          a.*, 
                          b.xrd, b.xsga, b.cogs, b.rdip, b.at, b.capx, b.ppegt, b.ppent, b.dp
                    FROM {LIBRARY}.{TABLE} AS a
                    LEFT JOIN comp.funda AS b ON a.gvkey = b.gvkey AND a.datadate = b.datadate
                    INNER JOIN {LINK_LIBRARY}.{LINK_TABLE} AS c ON a.gvkey = c.gvkey 
                    WHERE a.datadate BETWEEN c.linkdt AND COALESCE(c.linkenddt, CURRENT_DATE)
                            AND c.linktype IN ('LU','LC') AND c.linkprim IN ('P','C')
                """
    if start_date is not None: sql_string += r" AND a.datadate >= %(start_date)s"
    if end_date is not None: sql_string += r" AND a.datadate <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"
    
    return wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'nrows':nrows})

In [ ]:
#| eval: false
raw = get_raw_data(vars='*')
raw.head()

,permno,permco,liid,linkprim,gvkey,datadate,fyear,k_int_know,k_int_org,k_int_offbs,...,q_tot,xrd,xsga,cogs,rdip,at,capx,ppegt,ppent,dp
0,25881.0,23369.0,01,P,001000,1970-12-31,1970.0,0.0,5.328974,5.328974,...,0.817259,NaN,9.420,30.529,NaN,33.450,2.767,14.517,8.876,1.352
1,25881.0,23369.0,01,P,001000,1971-12-31,1971.0,0.0,7.607690,7.607690,...,0.508109,NaN,10.548,33.973,NaN,29.330,1.771,13.269,7.639,1.399
2,25881.0,23369.0,01,P,001000,1971-12-31,1971.0,0.0,7.607690,7.607690,...,0.508109,NaN,NaN,23.380,NaN,NaN,NaN,NaN,NaN,0.868
3,25881.0,23369.0,01,P,001000,1972-12-31,1972.0,0.0,8.550650,8.550650,...,0.516247,NaN,7.551,22.702,NaN,19.907,1.254,11.709,7.013,1.200
4,25881.0,23369.0,01,P,001000,1972-12-31,1972.0,0.0,8.550650,8.550650,...,0.516247,NaN,NaN,21.444,NaN,NaN,NaN,NaN,NaN,0.933


In [ ]:
#| export
def process_raw_data(
        df: pd.DataFrame=None,  # Must contain `permno` and `datadate` columns   
        clean_kwargs: dict={},  # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`"""

    # Change some variables to categorical
    for col in ['permno','permco']:
        if col in df.columns:
            df[col] = df[col].astype('Int64').astype('category')

    for col in ['gvkey']:
        if col in df.columns:
            df[col] = df[col].astype('string').astype('category')
            
    # Set up panel structure
    df = pdm.setup_panel(df, panel_ids=ENTITY_ID_IN_RAW_DSET, time_var=TIME_VAR_IN_RAW_DSET, freq=FREQ, panel_ids_toint=False, **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df_clean = process_raw_data(raw)

In [ ]:
#| export
def features(df: pd.DataFrame=None
             ) -> pd.DataFrame:

    out = df.sort_index().copy()

    for x in ['xrd','xsga','rdip']: out[f'{x}0'] = out[x].fillna(0)

    out['sga'] = np.where(out['xrd0'].between(out['xsga0'],out['cogs']) 
                        ,out['xsga0'],
                        out['xsga0'] - out['xrd0'] - out['rdip0'])
    if False:
        for x in ['xrd','xsga']:
            out[f'{x}0'] = np.where(out[x].isnull() & out['at'].notnull(), 0, out[x])
            out[f'{x}0'] = np.where(out[f'{x}0'].isnull() & out['at'].isnull(), out[f'{x}0'].interpolate(), out[f'{x}0'])

        out['sga'] = np.where(out['xsga'].isnull() | out['xrd0'].between(out['xsga0'],out['cogs']) 
                            ,out['xsga0'].fillna(0),
                            out['xsga0'] - out['xrd0'] - out['rdip'].fillna(0))    

    #They use ppegt below. I believe it should be ppent.
    out['k_phy'] = out['ppent'] #+ pdm.rdiff(out['dp']) #we reconstruct gross ppe because ppegt has a lot of missing values
    out['k_tot'] = out['k_phy'] + out['k_int']

    out['i_phy'] = out['capx']
    out['i_int'] = out['xrd0'] + 0.3*out['sga']
    out['i_tot'] = out['i_phy'] + out['i_int']

    out['i2k_int'] = out['i_int'] / pdm.lag(out['k_tot'])
    out['i2k_phy'] = out['i_phy'] / pdm.lag(out['k_tot'])
    out['i2k_tot'] = out['i2k_int'] + out['i2k_phy']    

    out = out.replace([np.inf, -np.inf], np.nan)
    return out 


In [ ]:
#| eval: false
ftrs = features(df_clean)

In [ ]:
#| eval: false
pdm.wins(ftrs[['i2k_tot','i2k_phy','i2k_int']]).describe()

,i2k_tot,i2k_phy,i2k_int
count,175623.000000,175623.000000,203412.000000
mean,0.245097,0.119157,0.122255
std,0.225062,0.164408,0.124075
min,0.000340,0.000000,0.000000
25%,0.115130,0.028412,0.025475
50%,0.191825,0.068188,0.100432
75%,0.294431,0.139660,0.172285
max,1.453925,1.066099,0.686001


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()